<a href="https://colab.research.google.com/github/LNickelsburg/clusterify/blob/main/apiify.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install spotipy

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd
import numpy as np
import glob

import spotipy
from spotipy.oauth2 import SpotifyOAuth
from spotipy.oauth2 import SpotifyClientCredentials

In [ ]:
SPOTIPY_CLIENT_ID='5259ed361642474a9ed685aa905cf558'
SPOTIPY_CLIENT_SECRET='303a3f63768b4cc5823497aeb06caa46'
SPOTIPY_REDIRECT_URI='https://example.com/callback/'
SCOPE="user-top-read"

sp=spotipy.Spotify(auth_manager=SpotifyOAuth(client_id=SPOTIPY_CLIENT_ID,
                                             client_secret=SPOTIPY_CLIENT_SECRET,
                                             redirect_uri=SPOTIPY_REDIRECT_URI,
                                             scope=SCOPE,

                                             open_browser=False))

In [ ]:
columns=['id','users','title','artist',
         'release_date','acousticness','danceability','duration_ms','energy',
         'explicit','instrumentalness','key','liveness','loudness','popularity',
         'mode','speechiness','tempo','time_signature','valence']

In [ ]:
tracks = pd.DataFrame(columns=columns)

# read all csvs to a dataframe of ids, labeled by user

directory = "/content/drive/MyDrive/Colab Notebooks/spotify_app/top_songs/"

for file_name in glob.glob(directory + '*.csv'):

    old_column = pd.read_csv(file_name, header=None)
    user = old_column.iloc[0, 0]
    users_songs = old_column.iloc[1, 0].split(',')

    for song in users_songs:
        song = song.replace("'", "").replace("[", "").replace("]", "").replace(" ", "")
        if song in tracks['id'].values:
            track_index = tracks[tracks['id']==song].index[0]
            track_users = tracks.at[track_index,'users']
            updated_users = track_users + "," + user
            tracks.at[track_index, 'users'] = updated_users
        else:
            new_row = pd.DataFrame({'id': [song], 'users': [user]})
            tracks = pd.concat([tracks, new_row], ignore_index=True)


In [ ]:
#break into chunks, for fewer api calls
chunker= lambda y, x: [y[i : i + x] for i in range(0, len(y), x)]
id_chunks = chunker
id_list=[]
for _, track in tracks.iterrows():
    id_list.append(track['id'])
id_chunks = chunker(id_list, 50)

In [ ]:
# track info, by chunk

# this is a big call to the API - make sure you only have to run this cell once
  # (otherwise you'll probs run into rate limit issues)

track_chunks = []
for i, chunk in enumerate(id_chunks):
    track_chunk = sp.tracks(chunk)
    track_chunks.append(track_chunk)



In [ ]:
#for each track in the chunk
i=0
for track_chunk in track_chunks:
    for track_info in track_chunk['tracks']:

        # artist
        artist_names = ', '.join([artist['name'] for artist in track_info['artists']])
        tracks.loc[i, 'artist'] = artist_names


        # other track info
        tracks.loc[i, 'release_date'] = track_info['album']['release_date']
        tracks.loc[i, 'duration_ms'] = track_info['duration_ms']
        tracks.loc[i, 'explicit'] = track_info['explicit']
        tracks.loc[i, 'popularity'] = track_info['popularity']
        tracks.loc[i, 'title'] = track_info['name']

        i+=1

In [ ]:
# audio features, by chunk
i=0
for chunk in id_chunks:
    feature_chunk = sp.audio_features(chunk)

    #for each track in the chunk
    for feature_info in feature_chunk:


        tracks.loc[i, 'acousticness'] = feature_info['acousticness']
        tracks.loc[i, 'danceability'] = feature_info['danceability']
        tracks.loc[i, 'energy'] = feature_info['energy']
        tracks.loc[i, 'instrumentalness'] = feature_info['instrumentalness']
        tracks.loc[i, 'key'] = feature_info['key']
        tracks.loc[i, 'liveness'] = feature_info['liveness']
        tracks.loc[i, 'loudness'] = feature_info['loudness']
        tracks.loc[i, 'mode'] = feature_info['mode']
        tracks.loc[i, 'speechiness'] = feature_info['speechiness']
        tracks.loc[i, 'tempo'] = feature_info['tempo']
        tracks.loc[i, 'time_signature'] = feature_info['time_signature']
        tracks.loc[i, 'valence'] = feature_info['valence']

        i+=1


In [ ]:
tracks.to_csv('/content/drive/MyDrive/Colab Notebooks/spotify_app/dataset.csv', index=False)